In [82]:
!pip install yfinance

In [83]:
from datetime import datetime
import numpy as np
import pandas as pd
import yfinance as yf
from sklearn.model_selection import train_test_split
import xgboost as xgb
import plotly.express as px
import plotly.graph_objects as go

In [84]:
symbol = "USDPKR=X"
date_today = str(datetime.now().date())
data = yf.download(symbol, interval="1d", start="1900-01-01",end=date_today)

[*********************100%***********************]  1 of 1 completed


In [85]:
features_df = pd.DataFrame()
features_df['dates'] = data.index
features_df['years'] = features_df['dates'].dt.year
features_df['months'] = features_df['dates'].dt.month
features_df['days'] = features_df['dates'].dt.day
features_df['day_week'] = features_df['dates'].dt.weekday
features_df['day_year'] = features_df['dates'].dt.dayofyear
features_df['timestamp'] = features_df['dates'].astype(int) // 10**11

In [86]:
X = features_df.drop('dates',axis=1).to_numpy()
y = data[['Close','High','Low','Open']].to_numpy()

In [87]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=42)

In [88]:
reg = xgb.XGBRegressor(random_state=42)
reg.fit(X_train,y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=None, n_jobs=None,
             num_parallel_tree=None, random_state=42, ...)

In [89]:
print("Train Score:{}".format(reg.score(X_train,y_train)))
print("Test Score:{}".format(reg.score(X_test,y_test)))

Train Score:0.9999761938701073
Test Score:0.9996157502806396


In [90]:
reg.predict(X_test)

array([[ 96.82307 , 104.517365, 103.02014 , 103.17004 ],
       [152.9516  , 152.94876 , 151.67705 , 152.21089 ],
       [ 90.219246,  90.71368 ,  89.32928 ,  89.4481  ],
       ...,
       [ 83.42759 ,  83.90294 ,  82.66122 ,  82.77331 ],
       [ 99.65538 , 100.06684 ,  99.90541 ,  99.281555],
       [101.640144, 101.68633 , 101.32837 , 101.39032 ]], dtype=float32)

In [91]:
def predict_vals(year,month,day,reg):
  """Date in the format of year,month,day"""
  dt = "{}-{}-{}".format(year,month,day)
  dt = pd.to_datetime(dt,format="%Y-%m-%d")

  weekday = dt.weekday()
  day_of_year = dt.dayofyear
  timestamp = int(dt.value)//10**11

  arr = np.array([[year,month,day,weekday,day_of_year,timestamp]])

  return reg.predict(arr)

In [92]:
import pandas as pd

def create_features_from_dates(dates):
    """
    Create features from a list or Series of datetime dates.

    Args:
        dates (list or pd.Series): List or Series of datetime dates.

    Returns:
        pd.DataFrame: A DataFrame with features derived from the dates.
    """
    # Convert dates to DataFrame
    dates_df = pd.DataFrame({"dates": dates})

    # Extract features
    dates_df['years'] = dates_df['dates'].dt.year
    dates_df['months'] = dates_df['dates'].dt.month
    dates_df['days'] = dates_df['dates'].dt.day
    dates_df['day_week'] = dates_df['dates'].dt.weekday  # Monday=0, Sunday=6
    dates_df['day_year'] = dates_df['dates'].dt.dayofyear
    dates_df['timestamp'] = dates_df['dates'].astype('int') // 10**11
    dates_df = dates_df.drop('dates',axis=1).to_numpy() # Unix timestamp in seconds

    return dates_df

In [93]:
future_dates = pd.to_datetime([features_df.iloc[-1]['dates'] + pd.Timedelta(days=x) for x in range(1,100)])

future_features = create_features_from_dates(future_dates)

predicted_vals = reg.predict(future_features)

In [94]:
predict_df = pd.DataFrame(predicted_vals,columns=['Close','High','Low','Open'],index=future_dates)
predict_df

,Close,High,Low,Open
2024-11-30 00:00:00+00:00,277.115540,276.471924,276.284485,276.154480
2024-12-01 00:00:00+00:00,277.924255,277.499878,274.806915,275.846313
2024-12-02 00:00:00+00:00,276.486237,277.941345,275.303802,275.244568
2024-12-03 00:00:00+00:00,278.424988,277.110046,275.705231,275.172363
2024-12-04 00:00:00+00:00,278.175262,277.288879,275.675446,275.272217
...,...,...,...,...
2025-03-04 00:00:00+00:00,277.527130,277.455322,276.688843,275.800629
2025-03-05 00:00:00+00:00,277.235565,276.903534,276.894501,276.451874
2025-03-06 00:00:00+00:00,278.057678,276.994293,275.616150,276.319244
2025-03-07 00:00:00+00:00,279.350464,277.628723,276.551758,276.949799


In [95]:
predict_vals(2025,5,30,reg)

array([[278.3669 , 276.69788, 276.9244 , 276.95752]], dtype=float32)

In [96]:
fig = go.Figure()

# Add traces for each series
fig.add_trace(go.Scatter(x=future_dates, y=predict_df['Open'], mode="lines", name="Open"))
fig.add_trace(go.Scatter(x=future_dates, y=predict_df['High'], mode="lines", name="High"))
fig.add_trace(go.Scatter(x=future_dates, y=predict_df['Low'], mode="lines", name="Low"))
fig.add_trace(go.Scatter(x=future_dates, y=predict_df['Close'], mode="lines", name="Close"))


# Update layout
fig.update_layout(
    title="Dollar to PKR prices for the next 100 days",
    xaxis_title="Dates",
    yaxis_title="Prices",
    legend_title="OHLC",
    template='plotly_dark'
)

# Show plot
fig.show()

In [97]:
import yfinance as yf

ticker_symbol = "AAPL"

stock_data = yf.Ticker(ticker_symbol)

history = stock_data.history(period="max")

In [98]:
features_df1 = pd.DataFrame()
features_df1['dates'] = history.index
features_df1['years'] = features_df1['dates'].dt.year
features_df1['months'] = features_df1['dates'].dt.month
features_df1['days'] = features_df1['dates'].dt.day
features_df1['day_week'] = features_df1['dates'].dt.weekday
features_df1['day_year'] = features_df1['dates'].dt.dayofyear
features_df1['timestamp'] = features_df1['dates'].astype(int) // 10**11

In [99]:
X1 = features_df1.drop('dates',axis=1).to_numpy()
y1 = history[['Open','High','Low','Close']].to_numpy()

In [100]:
X_train,X_test,y_train,y_test = train_test_split(X1,y1,test_size=0.2,random_state=42)

In [101]:
reg1 = xgb.XGBRegressor(random_state=42)
reg1.fit(X_train,y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=None, n_jobs=None,
             num_parallel_tree=None, random_state=42, ...)

In [102]:
print("Train Score: {}".format(reg1.score(X_train,y_train)))
print("Test Score: {}".format(reg1.score(X_test,y_test)))
preds = reg1.predict(X_test)

Train Score: 0.9999584042846721
Test Score: 0.9995852556934673


In [103]:
future_dates1 = pd.to_datetime([features_df1.iloc[-1]['dates'] + pd.Timedelta(days=x) for x in range(1,100)])

future_features1 = create_features_from_dates(future_dates1)

future_preds = reg1.predict(future_features1)

In [104]:
predict_df1 = pd.DataFrame(future_preds,columns=['Close','High','Low','Open'],index=future_dates1)
predict_df1

,Close,High,Low,Open
2024-11-30 00:00:00-05:00,234.670792,237.579376,233.300323,236.134705
2024-12-01 00:00:00-05:00,223.929230,230.403244,222.430008,229.942841
2024-12-02 00:00:00-05:00,222.765503,227.612122,222.041534,227.775604
2024-12-03 00:00:00-05:00,223.352219,230.210175,223.310760,228.509003
2024-12-04 00:00:00-05:00,223.401703,230.702637,222.969223,228.856537
...,...,...,...,...
2025-03-04 00:00:00-05:00,190.775070,193.926971,187.014679,185.134171
2025-03-05 00:00:00-05:00,190.095047,192.051910,186.684601,183.090530
2025-03-06 00:00:00-05:00,190.872894,192.598251,186.384216,183.310745
2025-03-07 00:00:00-05:00,186.892899,191.774231,185.513611,183.117645


In [105]:
predict_vals(2026,6,23,reg=reg1)

array([[211.99606, 219.63687, 209.44885, 213.9612 ]], dtype=float32)

In [106]:
fig1 = go.Figure()

# Add traces for each series
fig1.add_trace(go.Scatter(x=future_dates1, y=predict_df1['Open'], mode="lines", name="Open"))
fig1.add_trace(go.Scatter(x=future_dates1, y=predict_df1['High'], mode="lines", name="High"))
fig1.add_trace(go.Scatter(x=future_dates1, y=predict_df1['Low'], mode="lines", name="Low"))
fig1.add_trace(go.Scatter(x=future_dates1, y=predict_df1['Close'], mode="lines", name="Close"))


# Update layout
fig1.update_layout(
    title="Stock prices for the next 100 days",
    xaxis_title="Dates",
    yaxis_title="Prices",
    legend_title="OHLC",
    template='plotly_dark'
)

# Show plot
fig1.show()